In [1]:
# !pip install jsonlines

In [1]:
import jsonlines
import pandas as pd
import numpy as np

In [2]:
# text = []

# n = 0
# with jsonlines.open(r"ranking_train.jsonl", 'r') as reader:
#     for obj in  reader:
#         n += 1
#         text.append(obj)        
#         if n == 100:
#             break

# pd_data = pd.DataFrame(columns = ['post', 'comment', 'score'])
# i=0
# for body in text:
#     for comment in body['comments']:
#         pd_data.loc[i] = [body['text'].lower(), comment['text'].lower(), comment['score']]
#         i+=1

# pd_data

,post,comment,score
0,how many summer y combinator fundees decided n...,going back to school is not identical with giv...,0
1,how many summer y combinator fundees decided n...,there will invariably be those who don't see t...,1
2,how many summer y combinator fundees decided n...,for me school is a way to be connected to what...,2
3,how many summer y combinator fundees decided n...,i guess it really depends on how hungry you ar...,3
4,how many summer y combinator fundees decided n...,i know pollground decided to go back to school...,4
...,...,...,...
495,web4.0 -- the semantic web,i like the note about privacy. i believe it --...,0
496,web4.0 -- the semantic web,tbl has been touting the coming of a semantic ...,1
497,web4.0 -- the semantic web,the world godin presents is definitely scary f...,2
498,web4.0 -- the semantic web,some of the example applications he gave kick ...,3


In [3]:
# import from csv to df
pd_data = pd.read_csv('data/DataFrame.csv')
pd_data

,post,comment,score
0,how many summer y combinator fundees decided n...,going back to school is not identical with giv...,0
1,how many summer y combinator fundees decided n...,there will invariably be those who don't see t...,1
2,how many summer y combinator fundees decided n...,for me school is a way to be connected to what...,2
3,how many summer y combinator fundees decided n...,i guess it really depends on how hungry you ar...,3
4,how many summer y combinator fundees decided n...,i know pollground decided to go back to school...,4
...,...,...,...
278477,what am i missing about qr codes?,&#62;do any of you really whip out your smart ...,2
278478,what am i missing about qr codes?,qr codes as standardized visual targets for co...,3
278479,what am i missing about qr codes?,here is a thoughtful implementation of there u...,4
278480,dependency injection is a virtue,couldn't agree more. if the behaviour or outco...,0


In [12]:
from transformers import pipeline

In [81]:
df_comment = pd.DataFrame(columns=['comment'])
df_post = pd.DataFrame(columns=['post'])

In [88]:
def get_features(pd_data, df_base, clf, to_analyze="comment", num_of_comments=1000, batch_size=500):

    labels = []
    for n in range(int(num_of_comments/batch_size)):
        # if to_analyze == "comment":
        #     inbatch_step = 1
        # else:
        #     inbatch_step = 5
        pd_batch = pd_data[n*batch_size:(n+1)*batch_size]
        comments = list(pd_batch[to_analyze])
        result = clf(comments)
        if n == 0:
            for _dict in result[0]:
                labels.append(_dict['label'])
            df = pd.DataFrame(columns=labels)

        for i, _list in enumerate(result):
            for _dict in _list:
                # add to row i in column _dict['label'] value _dict['score']
                df.loc[n*batch_size + i, _dict['label']] = _dict['score']
    
    df = pd.concat([df_base, df], axis=1)
    return df
      

In [85]:
emotions_clf = pipeline(
    task = 'sentiment-analysis', 
    model = 'SamLowe/roberta-base-go_emotions', 
    tokenizer = 'SamLowe/roberta-base-go_emotions', 
    max_length=512, truncation=True, top_k=None)

In [89]:
df_comment = get_features(pd_data, df_comment, emotions_clf, to_analyze="comment", num_of_comments=1000, batch_size=500)

In [90]:
df_post = get_features(pd_data, df_post, emotions_clf, to_analyze="post", num_of_comments=1000, batch_size=500)

In [91]:
df_diff = df_post - df_comment
# add "diff" to column names
df_diff.columns = [str(col) + '_diff' for col in df_diff.columns]

In [92]:
# concat df and df_diff to one df
df_full = pd.concat([df_comment, df_diff], axis=1)

In [ ]:
# create nn model for classification of comments as input use df with 500 rows and 28 columns (28 emotions)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix
import warnings
from sklearn.exceptions import ConvergenceWarning

num_of_comments = 1000

X = df_full
y = pd_data['score'][:num_of_comments]

X_train, X_test, y_train, y_test = train_test_split(X, y)

scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

mlp = MLPClassifier(hidden_layer_sizes=(56, 56, 56, 56), max_iter=500, activation='relu', solver='adam', random_state=42)

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=ConvergenceWarning,
                            module="sklearn")
    
mlp.fit(X_train,y_train)

predictions = mlp.predict(X_test)

# print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.17      0.14      0.15        57
           1       0.14      0.18      0.15        45
           2       0.09      0.10      0.09        42
           3       0.19      0.17      0.18        53
           4       0.14      0.13      0.14        53

    accuracy                           0.14       250
   macro avg       0.14      0.14      0.14       250
weighted avg       0.15      0.14      0.14       250



In [40]:
# use grid search to find best activation function and solver
from sklearn.model_selection import GridSearchCV

parameter_space = {
    'hidden_layer_sizes': [(56, 56, 56, 56), (28, 28, 28, 28), (14, 14, 14, 14), (7, 7, 7, 7)],
    'activation': ['tanh', 'relu', 'logistic', 'identity'],
    'solver': ['sgd', 'adam'],
    # 'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

mlp = MLPClassifier()

clf = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=3)
clf.fit(X_train, y_train)

# Best paramete set
print('Best parameters found:\n', clf.best_params_)
# All results
means = clf.cv_results_['mean_test_score']

Best parameters found:
 {'activation': 'tanh', 'hidden_layer_sizes': (7, 7, 7, 7), 'solver': 'sgd'}
